In [46]:
import requests
import re
import pandas as pd
import json
from datetime import datetime
from lxml import html
from pprint import pprint
from pymongo import MongoClient

In [44]:
class YandexRuParser:
    host = 'https://yandex.ru/news'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) ' +
                             'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}

    def parse(self) -> list:
        news_list = []
        response = requests.get(self.host, headers=self.headers)
        dom = html.fromstring(response.text)
        news_elements = dom.xpath("//div[contains(@class,'mg-grid__row mg-grid__row_gap_8 news-top-flexible-stories news-app__top')]/div")
        for element in news_elements:
            news = {
                'title': str(element.xpath(".//h2/text()")[0]),
                'url':  str(element.xpath(".//a[@class='mg-card__link']/@href")[0]),
                'source': str(element.xpath(".//span[@class='mg-card-source__source']/a/text()")[0]),
                'publication_date': str(element.xpath(".//span[@class='mg-card-source__time']/text()")[0])
            }
            news_list.append(news)
        return news_list


if __name__ == '__main__':
    parser = YandexRuParser()
    news_result = parser.parse()
    pprint(news_result)

[{'publication_date': '15:20',
  'source': 'ТАСС',
  'title': 'Юлия Пересильд и Владимир Машков расписались на\xa0спускаемом '
           'аппарате «Союза МС-18»',
  'url': 'https://yandex.ru/news/story/YUliya_Peresild_i_Vladimir_Mashkov_raspisalis_naspuskaemom_apparate_Soyuza_MS-18--d5c74e373126a2ebbb3dba1775687387?lang=ru&rubric=index&fan=1&stid=ECsOiEWY9XTDVw_K5rEh&t=1634473796&tt=true&persistent_id=4666199945189050626'},
 {'publication_date': '15:24',
  'source': 'РБК',
  'title': 'Песков заявил об отсутствии перспектив для\xa0разговора Путина и '
           'Зеленского',
  'url': 'https://yandex.ru/news/story/Peskov_zayavil_ob_otsutstvii_perspektiv_dlyarazgovora_Putina_i_Zelenskogo--d5a59e043e5f37466cfceef2d5becaf6?lang=ru&rubric=index&fan=1&stid=ERwpy4_z04EY9wFF1FHS&t=1634473796&tt=true&persistent_id=4666199945189137406'},
 {'publication_date': '15:21',
  'source': 'Советский спорт',
  'title': 'Исмаилов обдумает возможность третьего боя с\xa0Минеевым после '
           'восстано

In [28]:
class MailRuParser:
    host = 'https://news.mail.ru/'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) ' +
                             'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}

    def parse(self) -> list:
        news_list = []
        response = requests.get(self.host, headers=self.headers)
        dom = html.fromstring(response.text)
        news_elements = dom.xpath("//ul[@data-module='TrackBlocks']/li[@class='list__item']")
        for element in news_elements:
            news = {
                'title': str(element.xpath(".//text()")[0]),
                'url': str(element.xpath("./a/@href")[0])
            }
            news = self._extend(news)
            news_list.append(news)
        return news_list

    def _extend(self, news: dict) -> dict:
        response = requests.get(news['url'], headers=self.headers)
        dom = html.fromstring(response.text)
        news['source'] = str(dom.xpath("//a[contains(@class,'breadcrumbs__link')]//text()")[0])
        news['publication_date'] = str(dom.xpath("//span[@datetime]/@datetime")[0])
        return news


if __name__ == '__main__':
    parser = MailRuParser()
    news_result = parser.parse()
    pprint(news_result)


[{'publication_date': '2021-10-17T12:06:07+03:00',
  'source': 'Life.ru',
  'title': 'В России хотят запретить остановку на\xa0«островках безопасности»',
  'url': 'https://news.mail.ru/society/48381363/'},
 {'publication_date': '2021-10-17T00:42:45+03:00',
  'source': 'Смотрим',
  'title': 'Кабо-Верде выдала США венесуэльского дипломата',
  'url': 'https://news.mail.ru/incident/48379037/'},
 {'publication_date': '2021-10-17T10:55:27+03:00',
  'source': 'РБК',
  'title': 'МВД объяснило новый порядок выезда детей за\xa0границу',
  'url': 'https://news.mail.ru/society/48381361/'},
 {'publication_date': '2021-10-17T11:03:11+03:00',
  'source': 'Новости Mail.ru',
  'title': 'Названы 5 фактов о\xa0Кремлевском дворце, которые стоит узнать',
  'url': 'https://news.mail.ru/society/48381364/'},
 {'publication_date': '2021-10-17T12:01:32+03:00',
  'source': 'Новости Mail.ru',
  'title': 'В\xa0Московском зоопарке показали лучших самцов-родителей (фото)',
  'url': 'https://news.mail.ru/society/4838

In [29]:
class LentaRuParser:
    host = 'https://lenta.ru'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) ' +
                             'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}

    def parse(self) -> list:
        news_list = []
        response = requests.get(self.host, headers=self.headers)
        dom = html.fromstring(response.text)
        news_elements = dom.xpath("//section[contains(@class,'js-top-seven')]//div[@class='item']")
        for element in news_elements:
            news = {
                'title': str(element.xpath("./a/text()")[0]),
                'url':  self.host + str(element.xpath("./a/@href")[0]),
                'source': 'LENTA_RU',
                'publication_date': str(element.xpath("./a/time/@datetime")[0])
            }
            news_list.append(news)
        return news_list


if __name__ == '__main__':
    parser = LentaRuParser()
    news_result = parser.parse()
    pprint(news_result)

[{'publication_date': ' 14:45, 17 октября 2021',
  'source': 'LENTA_RU',
  'title': 'Московские рестораны возьмут на\xa0работу роботов\xa0',
  'url': 'https://lenta.ruhttps://moslenta.ru/news/moskovskie-restorany-vozmut-na-rabotu-robotov-17-10-2021.htm?utm_source=from_lenta'},
 {'publication_date': ' 14:45, 17 октября 2021',
  'source': 'LENTA_RU',
  'title': 'Москвичей предупредили о «погодной лихорадке»',
  'url': 'https://lenta.ru/news/2021/10/17/pogoda/'},
 {'publication_date': ' 14:42, 17 октября 2021',
  'source': 'LENTA_RU',
  'title': 'Американская телеведущая сделала комплимент Путину',
  'url': 'https://lenta.ru/news/2021/10/17/compliment/'},
 {'publication_date': ' 14:34, 17 октября 2021',
  'source': 'LENTA_RU',
  'title': 'В Израиле предъявили обвинение главарю русской мафии',
  'url': 'https://lenta.ru/news/2021/10/17/israel/'},
 {'publication_date': ' 14:28, 17 октября 2021',
  'source': 'LENTA_RU',
  'title': 'Минеев объяснил «бешеную функционалку» Исмаилова допингом',


In [47]:
client = MongoClient('127.0.0.1', 27017)
db = client['news']

news_result = LentaRuParser().parse()
news_result += YandexRuParser().parse()
news_result += MailRuParser().parse()


db.news.insert_many(news_result)

pprint(news_result)

[{'_id': ObjectId('616c18ba78912609c837ecab'),
  'publication_date': ' 15:32, 17 октября 2021',
  'source': 'LENTA_RU',
  'title': 'Свидетельница конфликта в\xa0московском метро пообещала '
           'пожаловаться на\xa0полицейских',
  'url': 'https://lenta.ru/news/2021/10/17/metro_conflict/'},
 {'_id': ObjectId('616c18ba78912609c837ecac'),
  'publication_date': ' 15:26, 17 октября 2021',
  'source': 'LENTA_RU',
  'title': 'Россия депортировала племянника начальника Генштаба афганской '
           'армии',
  'url': 'https://lenta.ru/news/2021/10/17/deport/'},
 {'_id': ObjectId('616c18ba78912609c837ecad'),
  'publication_date': ' 15:25, 17 октября 2021',
  'source': 'LENTA_RU',
  'title': 'Нурмагомедов поддержал Исмаилова после нокаута от\xa0Минеева',
  'url': 'https://lenta.ru/news/2021/10/17/nurmagomedov/'},
 {'_id': ObjectId('616c18ba78912609c837ecae'),
  'publication_date': ' 15:22, 17 октября 2021',
  'source': 'LENTA_RU',
  'title': 'В Москве умер бывший сенатор от\xa0Астраханско